In [160]:
%load_ext Cython


The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [309]:
%%cython
import numpy as np
cimport numpy as np

cdef class TAHelper:
    cdef int i
    cdef int length
    cdef np.ndarray high
    cdef np.ndarray low
    cdef np.ndarray close
    cdef np.ndarray vol
    
    # 计算DX需要的变量
    cdef int dx_n
    cdef np.ndarray deltahigh
    cdef np.ndarray deltalow
    cdef np.ndarray plusdm
    cdef np.ndarray minusdm
    cdef np.ndarray admplus
    cdef np.ndarray admminus
    cdef np.ndarray tr
    cdef np.ndarray dx_atr
    cdef np.ndarray plusdi
    cdef np.ndarray minusdi
    cdef np.ndarray dx
    
    # 计算ADX需要的变量
    cdef np.ndarray adx
    
    # 计算MFI需要的变量
    cdef int mfi_n
    cdef np.ndarray typical_price
    cdef np.ndarray money_flow
    cdef np.ndarray positive_money_flow
    cdef np.ndarray negative_money_flow
    cdef np.ndarray money_ratio
    cdef np.ndarray mfi
    
    # 计算APO需要的变量
    cdef int apo_fast_n
    cdef int apo_slow_n
    cdef np.ndarray apo_fastma 
    cdef np.ndarray apo_slowma
    cdef np.ndarray apo

    # 计算MACD需要的变量
    cdef int macd_fast_n
    cdef int macd_slow_n
    cdef int macd_signal_n
    cdef np.ndarray macd_fastema 
    cdef np.ndarray macd_slowema
    cdef np.ndarray macd
    cdef np.ndarray macdsignal
    cdef np.ndarray macdhist
    
    # 计算NATR需要的变量
    cdef int natr_n
    cdef np.ndarray natr_atr
    cdef np.ndarray natr
    
    # 计算PLUS_DM需要的变量
    cdef int plus_dm_n
    cdef np.ndarray plus_dm
    cdef double init_plusdm
    
    # 计算RSI需要的变量
    cdef int rsi_n
    cdef np.ndarray upavg
    cdef np.ndarray dnavg
    cdef np.ndarray rsi
    
    # 计算TEMA需要的变量
    cdef int tema_n
    cdef np.ndarray tema_ema1
    cdef np.ndarray tema_ema2
    cdef np.ndarray tema_ema3
    cdef np.ndarray tema
    
    def __init__(self):
        self.i = -1
        self.length = 10000
        self.high = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.low = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.close = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.vol = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算DX需要的变量
        self.dx_n = 14
        self.deltahigh = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.deltalow = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.plusdm = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.minusdm = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.admplus = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.admminus = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.tr = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.dx_atr = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.plusdi = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.minusdi = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.dx = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算ADX需要的变量
        self.adx = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算MFI需要的变量
        self.mfi_n = 14
        self.typical_price = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.money_flow = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.positive_money_flow = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.negative_money_flow = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.money_ratio = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.mfi = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算APO需要的变量
        self.apo_fast_n = 12
        self.apo_slow_n = 26
        self.apo_fastma = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.apo_slowma = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.apo = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算MACD需要的变量
        self.macd_fast_n = 12
        self.macd_slow_n = 26
        self.macd_signal_n = 9
        self.macd_fastema = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.macd_slowema = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.macd = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.macdsignal = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.macdhist = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算NATR需要的变量
        self.natr_n = 14
        self.natr_atr = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.natr = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算PLUS_DM需要的变量
        self.plus_dm_n = 14
        self.plus_dm = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.init_plusdm = 0
        
        # 计算RSI需要的变量
        self.rsi_n = 14
        self.upavg = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.dnavg = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.rsi = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
        # 计算TEMA需要的变量
        self.tema_n = 30
        self.tema_ema1 = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.tema_ema2 = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.tema_ema3 = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        self.tema = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
    cdef double TrueRange(self, double high, double low, double preClose):
        return max([high-low, abs(high-preClose), abs(preClose-low)])
    
    cdef double DX_inc(self, double plusDI, double minusDI):
        return 100 * abs(plusDI - minusDI) / (plusDI + minusDI + np.finfo(float).eps)
    
    cdef double MFI_inc(self, double MoneyRatio):
        return 100 - (100/(1+MoneyRatio))
    
    cdef double APO_inc(self, double fastMA, double slowMA):    
        return fastMA - slowMA
    
    cdef np.ndarray MACD_inc(self, double fastema, double slowema, double preSignal, int signal_n):
        cdef double macd = fastema - slowema
        cdef double signal = ((signal_n-1)/(signal_n+1))*preSignal + (2/(signal_n+1))*macd
        cdef double hist = macd - signal
        return np.array([macd, signal, hist])
    
    cdef double NATR_inc(self, double close, double atr):
        cdef double natr = 100*atr/close
        return natr
    
    cdef np.ndarray dm_one_period(self, double high, double prehigh, double low, double prelow):
        cdef double deltahigh = high - prehigh
        cdef double deltalow = prelow - low
        cdef double plusdm
        cdef double minusdm

        if (deltahigh < 0 and deltalow < 0) or deltahigh == deltalow:
            plusdm = 0
            minusdm = 0
        elif deltahigh > deltalow:
            plusdm = deltahigh
            minusdm = 0
        else:
            plusdm = 0
            minusdm = deltalow

        return np.array([plusdm, minusdm])
    
    cdef double PLUS_DM_inc(self, double high, double prehigh, double low, double prelow, double pre_plus_dm_n, int n):
        plusdm, _ = self.dm_one_period(high, prehigh, low, prelow)
        plus_dm_n = pre_plus_dm_n - pre_plus_dm_n/n + plusdm

        return plus_dm_n
    
    cdef double RSI_inc(self, double close, double preClose, double preUpavg, double preDnavg, int n=14):
        cdef double up
        cdef double dn
        if close > preClose:
            up = close - preClose
            dn = 0
        else:
            up = 0
            dn = preClose - close

        upavg = (preUpavg*(n-1) + up) / n
        dnavg = (preDnavg*(n-1) + dn) / n
        rsi = 100 * upavg/(upavg + dnavg)

        return rsi
    
    cdef double TEMA_inc(self, double ema1, double ema2, double ema3):
        return 3*ema1 - 3*ema2 + ema3
 
    cdef dx_depends_update(self):
        # DX 依赖变量更新
        # 更新 typicalprice, moneyflow
        self.typical_price[self.i] = (self.high[self.i] + self.low[self.i] + self.close[self.i]) / 3
        self.money_flow[self.i] = self.typical_price[self.i] * self.vol[self.i]
        
        if self.i == 0:
            pass
        else:
            self.deltahigh[self.i] = self.high[self.i] - self.high[self.i-1]
            self.deltalow[self.i] = self.low[self.i-1] - self.low[self.i]
            self.tr[self.i] = self.TrueRange(self.high[self.i], self.low[self.i], self.close[self.i-1])
            # 更新 plusdm minusdm
            if (self.deltahigh[self.i] < 0 and self.deltalow[self.i] < 0) or \
            (self.deltahigh[self.i] == self.deltalow[self.i]):
                self.plusdm[self.i] = 0
                self.minusdm[self.i] = 0
            elif self.deltahigh[self.i] > self.deltalow[self.i]:
                self.plusdm[self.i] = self.deltahigh[self.i]
                self.minusdm[self.i] = 0
            elif self.deltahigh[self.i] < self.deltalow[self.i]:
                self.plusdm[self.i] = 0
                self.minusdm[self.i] = self.deltalow[self.i]
                
            if self.i == 1:
                # 更新第一个 adm
                
                self.admplus[self.i] = self.plusdm[self.i]
                self.admminus[self.i] = self.minusdm[self.i]
                
                # 更新第一个 atr
                self.dx_atr[self.i] = self.tr[self.i]  
            else:
                # 更新第二个及以后的 adm
                
                self.admplus[self.i] = (self.plusdm[self.i] + (self.dx_n-1)*self.admplus[self.i-1]) / self.dx_n
                self.admminus[self.i] = (self.minusdm[self.i] + (self.dx_n-1)*self.admminus[self.i-1]) / self.dx_n

                # 更新第二个及以后的 atr
                self.dx_atr[self.i] = (self.tr[self.i] + (self.dx_n-1)*self.dx_atr[self.i-1]) / self.dx_n
            # 更新 di
            self.plusdi[self.i] = 100 * self.admplus[self.i] / self.dx_atr[self.i]
            self.minusdi[self.i] = 100 * self.admminus[self.i] / self.dx_atr[self.i]
            # 更新 dx
            self.dx[self.i] = self.DX_inc(self.plusdi[self.i], self.minusdi[self.i])
            
            # 更新 adx
            if self.i == 1:
                self.adx[self.i] = self.dx[self.i]
            else:
                self.adx[self.i] = ((self.dx_n-1) * self.adx[self.i-1] + self.dx[self.i]) / self.dx_n
                
    cdef mfi_depends_update(self):
        # MFI 依赖变量更新
        # 更新 typicalprice, moneyflow
        self.typical_price[self.i] = (self.high[self.i] + self.low[self.i] + self.close[self.i]) / 3
        self.money_flow[self.i] = self.typical_price[self.i] * self.vol[self.i]

        if self.i == 0:
            pass
        else:
            # 更新 positivemoneyflow, negativemoneyflow
            if self.typical_price[self.i] > self.typical_price[self.i-1]:
                self.positive_money_flow[self.i] = self.money_flow[self.i]
                self.negative_money_flow[self.i] = 0
            else:
                self.positive_money_flow[self.i] = 0
                self.negative_money_flow[self.i] = self.money_flow[self.i]

            # 更新 moneyratio
            if self.i >= self.mfi_n:
                self.money_ratio[self.i] = ( self.positive_money_flow[self.i-self.mfi_n+1:self.i+1].sum() /                                                          self.negative_money_flow[self.i-self.mfi_n+1:self.i+1].sum() )
                # 更新 moneyflowindex
                self.mfi[self.i] = self.MFI_inc(self.money_ratio[self.i])
           
    cdef apo_depends_update(self):
        # APO 依赖变量更新
        # 更新fastma
        if self.i >= self.apo_fast_n-1:
            self.apo_fastma[self.i] = self.close[self.i-self.apo_fast_n+1:self.i+1].mean()
        # 更新slowma
        if self.i >= self.apo_slow_n-1:
            self.apo_slowma[self.i] = self.close[self.i-self.apo_slow_n+1:self.i+1].mean()
            # 更新apo
            self.apo[self.i] = self.APO_inc(self.apo_fastma[self.i], self.apo_slowma[self.i])
            
    cdef macd_depends_update(self):
        # MACD 依赖变量更新
        # 更新第一个fastema slowema
        if self.i == 0:
            self.macd_fastema[self.i] = self.close[self.i]
            self.macd_slowema[self.i] = self.close[self.i]
            self.macd[self.i] = self.macd_fastema[self.i] - self.macd_slowema[self.i]
            self.macdsignal[self.i] = self.macd[self.i]
            self.macdhist[self.i] = self.macd[self.i] - self.macdsignal[self.i]
        else:
            self.macd_fastema[self.i] = self.macd_fastema[self.i-1]*((self.macd_fast_n-1)/(self.macd_fast_n+1)) + (2/(self.macd_fast_n+1))*self.close[self.i]
            self.macd_slowema[self.i] = self.macd_slowema[self.i-1]*((self.macd_slow_n-1)/(self.macd_slow_n+1)) + (2/(self.macd_slow_n+1))*self.close[self.i]
            self.macd[self.i], self.macdsignal[self.i], self.macdhist[self.i] = self.MACD_inc(self.macd_fastema[self.i], self.macd_slowema[self.i], self.macdsignal[self.i-1], self.macd_signal_n)                
            
    cdef natr_depends_update(self):
        # NATR 依赖变量更新
        if self.i == 0:
            pass
        else:
            if self.i == 1:
                # 更新第一个natr_atr
                self.natr_atr[self.i] = self.tr[self.i] 
            elif self.i >= 2:
                self.natr_atr[self.i] = (self.tr[self.i] + (self.natr_n-1)*self.natr_atr[self.i-1]) / self.natr_n
                self.natr[self.i] = self.NATR_inc(self.close[self.i], self.natr_atr[self.i])
                
    cdef plus_dm_depends_update(self):
        # PLUS_DM 依赖变量更新
        if self.i == 0:
            pass
        elif self.i < self.plus_dm_n:
            self.init_plusdm += self.dm_one_period(self.high[self.i], self.high[self.i-1], self.low[self.i], self.low[self.i])[0]
            if self.i == self.plus_dm_n-1:
                self.plus_dm[self.i] = self.init_plusdm
        else:
            self.plus_dm[self.i] = self.PLUS_DM_inc(self.high[self.i], self.high[self.i-1], self.low[self.i], self.low[self.i-1], self.plus_dm[self.i-1], self.plus_dm_n) 
            
    cdef rsi_depends_update(self):
        # RSI 依赖变量更新
        if self.i == 0:
            pass
        else:
            if self.close[self.i] > self.close[self.i-1]:
                up = self.close[self.i] - self.close[self.i-1]
                dn = 0
            else:
                up = 0
                dn = self.close[self.i-1] - self.close[self.i]
                
            if self.i == 1:
                self.upavg[self.i] = up
                self.dnavg[self.i] = dn
            else:
                self.rsi[self.i] = self.RSI_inc(self.close[self.i], self.close[self.i-1], self.upavg[self.i-1], self.dnavg[self.i-1], self.rsi_n)
                self.upavg[self.i] = ((self.rsi_n-1)*self.upavg[self.i-1] + up) / self.rsi_n
                self.dnavg[self.i] = ((self.rsi_n-1)*self.dnavg[self.i-1] + dn) / self.rsi_n    
 
    cdef tema_depends_update(self):
        # TEMA 依赖变量更新
        if self.i == 0:
            # 更新第一个ema1 ema2 ema3
            self.tema_ema1[self.i] = self.close[self.i]
            self.tema_ema2[self.i] = self.tema_ema1[self.i]
            self.tema_ema3[self.i] = self.tema_ema2[self.i]
        else:
            self.tema_ema1[self.i] = ((self.tema_n-1)/(self.tema_n+1))*self.tema_ema1[self.i-1] + (2/(self.tema_n+1))*self.close[self.i]
            self.tema_ema2[self.i] = ((self.tema_n-1)/(self.tema_n+1))*self.tema_ema2[self.i-1] + (2/(self.tema_n+1))*self.tema_ema1[self.i]
            self.tema_ema3[self.i] = ((self.tema_n-1)/(self.tema_n+1))*self.tema_ema3[self.i-1] + (2/(self.tema_n+1))*self.tema_ema2[self.i]
            
        self.tema[self.i] = self.TEMA_inc(self.tema_ema1[self.i], self.tema_ema2[self.i], self.tema_ema3[self.i])  
          
    cpdef new_bar(self, double new_high, double new_low, double new_close, double new_vol):
        self.i += 1
        self.high[self.i] = new_high
        self.low[self.i] = new_low
        self.close[self.i] = new_close
        self.vol[self.i] = new_vol
        

        self.dx_depends_update()
        self.mfi_depends_update()
        self.apo_depends_update()
        self.macd_depends_update()
        self.plus_dm_depends_update()    
        self.rsi_depends_update()
        self.tema_depends_update()
        

        
    cpdef np.ndarray get_indicators(self):
        return np.array([
                self.dx[self.i],
                self.adx[self.i],
                self.mfi[self.i],
                self.apo[self.i],
                self.macd[self.i],
                self.macdsignal[self.i],
                self.macdhist[self.i],
                self.natr[self.i],
                self.plus_dm[self.i],
                self.rsi[self.i],
                self.tema[self.i]
        ]
        )

In [18]:
%time TAHelper()

AttributeError: '_cython_magic_1f1ae2bd46fb8013986b359dea862d5f.TAHelper' object has no attribute 'i'

In [76]:
%%cython
cimport numpy as np
import numpy as np

cdef np.ndarray high = np.full(10, fill_value=np.nan, dtype=np.float64)

In [106]:
%%cython

cdef int i

In [262]:
# test

import numpy as np 

high = np.random.random(1000)
low = np.random.random(1000)
close = np.random.random(1000)
vol = np.random.random(1000)


In [310]:
ta = TAHelper()

In [279]:
ta.new_bar(high[0], low[0], close[0], vol[0])

[0.6710728 0.6710728       nan ...       nan       nan       nan]


ZeroDivisionError: float division

a


Exception ignored in: '_cython_magic_bec79b8b9316bf05d14a8f8ad46169e1.TAHelper.DX_inc'
ZeroDivisionError: float division


In [304]:
for i in range(len(high)):
    ta.new_bar(high[i], low[i], close[i], vol[i])

plusdi:  [nan nan nan ... nan nan nan]
minusdi:  [nan nan nan ... nan nan nan]


ZeroDivisionError: float division

plusdi:  [nan  0. nan ... nan nan nan]
minusdi:  [nan  0. nan ... nan nan nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.28965056 ...        nan        nan        nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.28965056 ...        nan        nan        nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.28965056 ...        nan        nan        nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.28965056 ...        nan        nan        nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.28965056 ...        nan        nan        nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.28965056 ...        nan        nan        nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.28965056 ...        nan        nan        nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.

Exception ignored in: '_cython_magic_4596664b2f9d64e9a59cc7960ca782fb.TAHelper.DX_inc'
ZeroDivisionError: float division


 [       nan 0.         7.28965056 ...        nan        nan        nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.28965056 ...        nan        nan        nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.28965056 ...        nan        nan        nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.28965056 ...        nan        nan        nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.28965056 ...        nan        nan        nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.28965056 ...        nan        nan        nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.28965056 ...        nan        nan        nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.28965056 ...        nan        nan        nan]
plusdi:  [nan  0.  0. ... nan nan nan]
minusdi:  [       nan 0.         7.289650

In [298]:
ta.high

AttributeError: '_cython_magic_3f9011280cc320160e680e7cd3274a44.TAH' object has no attribute 'high'

In [311]:
ta = TAHelper()
DX = list()
ADX = list()
MFI = list()
APO = list()
MACD = list()
MACDsignal = list()
MACDhist = list()
NATR = list()
PLUS_DM = list()
RSI = list()
TEMA = list()

for i in range(len(high)):
    ta.new_bar(high[i], low[i], close[i], vol[i])
    indicators = ta.get_indicators()
    DX.append(indicators[0])
    ADX.append(indicators[1])
    MFI.append(indicators[2])
    APO.append(indicators[3])
    MACD.append(indicators[4])
    MACDsignal.append(indicators[5])
    MACDhist.append(indicators[6])
    NATR.append(indicators[7])
    PLUS_DM.append(indicators[8])
    RSI.append(indicators[9])
    TEMA.append(indicators[10])

In [312]:
import talib 
np.set_printoptions(threshold=200)

np.array(DX) - talib.DX(high, low, close)

array([            nan,             nan,             nan, ...,
       -2.13162821e-14, -1.42108547e-14, -1.77635684e-14])

In [313]:
np.array(ADX) - talib.ADX(high, low, close)

array([            nan,             nan,             nan, ...,
       -7.10542736e-15, -7.10542736e-15, -1.06581410e-14])

In [314]:
np.array(MFI) - talib.MFI(high, low, close, vol)

array([           nan,            nan,            nan, ...,
       2.84217094e-14, 2.84217094e-14, 4.26325641e-14])

In [315]:
np.array(APO) - talib.APO(close)

array([            nan,             nan,             nan, ...,
       -1.11022302e-15, -1.05471187e-15, -1.11022302e-15])

In [316]:
macd1 = np.array(MACD)
signal1 = np.array(MACDsignal)
hist1 = np.array(MACDhist)
macd2, signal2, hist2 = talib.MACD(close)

In [320]:
macd1 - macd2

array([nan, nan, nan, ...,  0.,  0.,  0.])

In [321]:
signal1 - signal2

array([            nan,             nan,             nan, ...,
       -6.59194921e-17, -5.29090660e-17, -4.16333634e-17])

In [322]:
hist1 - hist2

array([           nan,            nan,            nan, ...,
       6.93889390e-17, 5.29090660e-17, 4.16333634e-17])

In [324]:
np.set_printoptions(threshold=2000)
np.array(NATR) - talib.NATR(high, low, close)

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [231]:
%%cython
import numpy as np
cimport numpy as np


cdef class A:
    cdef np.ndarray high
    cdef int length
    
    def __init__(self):
        self.length = 10
        self.high = np.full(self.length, fill_value=np.nan, dtype=np.float64)
        
    cpdef func(self):
        for i in range(self.length):
            self.high[i] = i
            
    cpdef show(self):
        print(self.high)

In [232]:
a = A()

In [215]:
a.func()

In [233]:
a.show()

[nan nan nan nan nan nan nan nan nan nan]


In [306]:
np.finfo(float).eps

2.220446049250313e-16

In [326]:
round(12.6)

13